<a href="https://colab.research.google.com/github/Geopalut/AdversarialML/blob/master/basic_attacks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Attacking a basic convnet 

In this notebook I will experiment with different adversaries using a basic convolutionnal neural network trained on CIFAR10 as the target. The notebook is organised as follows:

1. Building and training a basic net  
2. Building adversarial examples using FGSM
3. Building adversarial examples using a multiple step PGD
4. Trying to improve the defense of the network using adversarial training. 


### 1. The target network

For this example I build a basic convolutional neural network with three convolutionnal layers and two dense layers. 
There is also some loght data processing to import CIFAR 10 images.